In [1]:
from redminelib import Redmine
import re
import sharepy
import json
from ipywidgets import widgets

In [2]:
lbl1 = widgets.Label("Sharepoint User Name")
sp_username = widgets.Text()
display(lbl1, sp_username)

lbl2 = widgets.Label("Redmine User Name")
rm_username = widgets.Text()
display(lbl2, rm_username)

lbl3 = widgets.Label("ÖAW Password")
pwd = widgets.Password()
display(lbl3, pwd)




Label(value='Sharepoint User Name')

Text(value='')

Label(value='Redmine User Name')

Text(value='')

Label(value='ÖAW Password')

Password()

In [3]:
## Redmine Setup 

redmine = Redmine('https://redmine.acdh.oeaw.ac.at', username=rm_username.value, password=pwd.value)#'c39a3d555a81559e0ea7edfcd77b5e7d92099c24')

RESOURCE_TRACKER_ID = 5
SERVICE_TRACKER_ID = 7
PROJECT_TRACKER_ID = 8

## Helper Functions 

def custom_field_id_by_name(name):
    all_fields = redmine.custom_field.all()
    return list(filter(lambda x: x['name']==name, all_fields))[0]['id']

def custom_field_by_name(issue, name, defaultValue="N/A"):
    cf = issue['custom_fields']
    v = list(filter(lambda f: f['name'] == name, cf))[0]['value']
    return defaultValue if v in [None, ""] else v
    
   
    
LOCATION_FIELD_ID = custom_field_id_by_name('location')
LOCATION_PATH_FIELD_ID = custom_field_id_by_name('location_path')
SIZE_FIELD_ID = custom_field_id_by_name('size')

In [4]:
# Sharepoint Configuration

SP_BASEURL = "oeawacat.sharepoint.com"
SP_NAME = "ACDH-CH_i_hub"
SP_PROJECT_LIST_NAME = "Projekte"
personsListName = "Personen"
s = sharepy.connect(SP_BASEURL, username=sp_username.value, password=pwd.value)

Authentication successful   


In [5]:
# Sharepoint pages long lists – this helper function  wraps the iteration
def getSPList(sp_name, sp_list_name):
    url = "https://"+SP_BASEURL+"/sites/"+sp_name+"/_api/lists/getbytitle('"+sp_list_name+"')/items"
    r = s.get(url).json()
    nextUrl = None
    listItems = r['d']['results']
    if '__next' in r['d'].keys():
        nextUrl = r['d']['__next']
    while nextUrl is not None:
        r = s.get(nextUrl).json()
        for i in r['d']['results']:
            listItems.append(i)
        if '__next' in r['d'].keys():
            nextUrl = r['d']['__next']
        else:
            nextUrl = None
            
    return listItems

In [7]:
# get all projects from Project List in Sharepoint,
# try to parse the Redmine Issue ID from the *RedmineLink* field 
# and store it in a simplified object.

projectsFromSharepoint = []
for proj in getSPList(SP_NAME, SP_PROJECT_LIST_NAME):
    redmineID = 0
    if proj.get('RedmineLink') is not None:
        RedmineLinkField = proj.get('RedmineLink').get('Description')
        try:
            redmineID = int(re.sub("[^0-9]+","", RedmineLinkField))
        except:
            print("WATCHME cannot parse "+RedmineLinkField+" as a redmine issue ID")
    p = {
        'id' : int(proj['Id']),
        'title' : proj['Title'],
        'shortTitle' : proj['ShortTitle'],
        'redmineID': redmineID
    }
    projectsFromSharepoint.append(p)
%store projectsFromSharepoint

WATCHME cannot parse PAAS#Project in Redmine as a redmine issue ID
WATCHME cannot parse https://redmine.acdh.oeaw.ac.at/projects/voice_clariah as a redmine issue ID
Stored 'projectsFromSharepoint' (list)


In [8]:
# Get every project issue from Redmine
# and try to match it against the projectlist 
# from Sharepoint

projectsFromRedmine = list(redmine.issue.filter(tracker_id = PROJECT_TRACKER_ID, include = ['relations']).values())

%store projectsFromRedmine

for rp in sorted(projectsFromRedmine, key=lambda x: x['subject']):
    id, subject = rp['id'], rp['subject']
    spProjectByID = list(filter(lambda x: x['redmineID'] == id, projectsFromSharepoint))
    if not spProjectByID:
        print("https://redmine.acdh.oeaw.ac.at/issues/" + str(id) + " '" + subject +  "' IS MISSING OR NOT LINKED in ProjektListe")

Stored 'projectsFromRedmine' (list)
https://redmine.acdh.oeaw.ac.at/issues/14587 '"Living in the Alps" für Informationsaufbereitung für die Digital Humanities (DI4DH)' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/10572 ''Die große Transformation'' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/6903 '(Infrastructure for) Russian manuscript lexicons - Kovalenko' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/19920 'ABCD - Anton Bruckner Chronologie Datenbank Projekt-Issue' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/6782 'ABLO – Bruckner-Dictionary' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/13501 'ACDH Universal Dashboard' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/837 'APIS (ÖAW0405)' IS MISSING OR NOT LINKED in ProjektListe
https://redmine.acdh.oeaw.ac.at/issues/11098 'APIS dev

In [5]:
#ss = redmine.issue.filter(tracker_id = RESOURCE_TRACKER_ID, include = ['relations']).vaalues()

```
--- cut here --
    from here on things get messy
    ```
   

In [12]:
rr = list(redmine.issue.filter(tracker_id = RESOURCE_TRACKER_ID, include = ['relations']).values())

In [7]:
filtered = filter(lambda x: x['project']['name'] not in ['acdh-hardware', 'dev-core'], rr)

In [8]:
shortened = []

In [9]:
for f in filtered: 
    size = custom_field_by_name(f, "size")
    format = custom_field_by_name(f, "format")
    location = custom_field_by_name(f, "location")
    location_path = custom_field_by_name(f, "location_path")
    
    # project might be parent issues or related issues
    rels = filter(lambda r: r['relation_type'] in ['parent', 'relates'], f['relations'])
    project = ""
    for rel in rels:
        tId = rel['issue_id']
        t = redmine.issue.get(tId)
        tType = t['tracker']['name']
        if tType == "Project":
            project = t['subject']
    
  #  otherwise, we fall back on the redmine issue 
    if project == "":
        project = f['project']['name']
        
    o = {
        'id': f['id'], 
        'status' : f['status']['name'],
        'subject': f['subject'],
        'size': size,
        'format': format,
        'location': location,
        'project': project,
        'location_path': location_path
    }
    shortened.append(o)


shortened

[{'id': 20167,
  'status': 'In Preparation',
  'subject': 'TUNOCENT Recordings list',
  'size': 'N/A',
  'format': 'Excel, (TEI)',
  'location': ['owncloud'],
  'project': 'TUNOCENT',
  'location_path': 'https://oeawcloud.oeaw.ac.at/index.php/f/37373125'},
 {'id': 20117,
  'status': 'New',
  'subject': 'DBÖ data @ ARCHE',
  'size': 'N/A',
  'format': 'N/A',
  'location': ['storage'],
  'project': 'ARCHE curation',
  'location_path': '/mnt/acdh_resources/ARCHE/staging/DBOE_20117'},
 {'id': 20065,
  'status': 'New',
  'subject': 'Kretschmer data',
  'size': 'N/A',
  'format': 'N/A',
  'location': ['storage'],
  'project': 'ARCHE curation',
  'location_path': 'ARCHE/staging/Kretschmer_20065/original'},
 {'id': 20063,
  'status': 'In Preparation',
  'subject': 'Bruckner Kopisten',
  'size': 'N/A',
  'format': 'TEI, images',
  'location': [],
  'project': ' DSIE - Data, Services, Infrastructure, Education ',
  'location_path': 'container/R_brucknerkopisten_20063'},
 {'id': 20015,
  'status'

In [30]:
filtered(lambda i: i['Subject'] shortened)

[]